In [1]:
import pandas as pd
import re
import os

In [2]:
def read(folder_path):
  log_arrays = {}
  for filename in os.listdir(folder_path):
    if filename.endswith('.log'):
      with open(os.path.join(folder_path, filename), 'r') as f:
        log_array = f.readlines()
        log_name = filename[:-7]
        log_arrays[log_name] = log_array
        print(log_name)
  return log_arrays

In [10]:
folder_path = "../raw_data"
log_arrays = read(folder_path)

Android
Apache
BGL
Hadoop
HDFS
HealthApp
HPC
Linux
Mac
OpenSSH
OpenStack
Proxifier
Spark
Thunderbird
Windows
Zookeeper


In [12]:
def clean_and_normalize_logs(logs):

    cleaned_logs = {}

    for source, log_array in logs.items():
        cleaned_log_array = []
        for log in log_array:
            cleaned_log = re.sub(r'[^\w\s]', '', log)
            cleaned_log = re.sub(r'\s+', ' ', cleaned_log).strip()
            cleaned_log_array.append(cleaned_log)

        cleaned_logs[source] = cleaned_log_array

    return cleaned_logs

logs = log_arrays
cleaned_logs = clean_and_normalize_logs(logs)